# Installation
Just pip install:

```pip install omegaconf```

If you want to try this notebook after checking out the repository be sure to run 
```python setup.py develop``` at the repository root before running this code.

# Creating OmegaConf objects
### Empty

In [1]:
from omegaconf import OmegaConf
conf = OmegaConf.create()
print(conf)

{}


### From a dictionary

In [2]:
conf = OmegaConf.create(dict(k='v',list=[1,dict(a='1',b='2')]))
print(OmegaConf.to_yaml(conf))

k: v
list:
- 1
- a: '1'
  b: '2'



### From a list

In [3]:
conf = OmegaConf.create([1, dict(a=10, b=dict(a=10))])
print(OmegaConf.to_yaml(conf))

- 1
- a: 10
  b:
    a: 10



### From a yaml file

In [4]:
conf = OmegaConf.load('../source/example.yaml')
print(OmegaConf.to_yaml(conf))

server:
  port: 80
log:
  file: ???
  rotation: 3600
users:
- user1
- user2



### From a yaml string

In [5]:
yaml = """
a: b
b: c
list:
- item1
- item2
"""
conf = OmegaConf.create(yaml)
print(OmegaConf.to_yaml(conf))

a: b
b: c
list:
- item1
- item2



### From a dot-list

In [6]:
dot_list = ["a.aa.aaa=1", "a.aa.bbb=2", "a.bb.aaa=3", "a.bb.bbb=4"]
conf = OmegaConf.from_dotlist(dot_list)
print(OmegaConf.to_yaml(conf))

a:
  aa:
    aaa: 1
    bbb: 2
  bb:
    aaa: 3
    bbb: 4



### From command line arguments

To parse the content of sys.arg:

In [7]:
# Simulating command line arguments
import sys
sys.argv = ['your-program.py', 'server.port=82', 'log.file=log2.txt']
conf = OmegaConf.from_cli()
print(OmegaConf.to_yaml(conf))

server:
  port: 82
log:
  file: log2.txt



# Access and manipulation
Input yaml file:

In [8]:
conf = OmegaConf.load('../source/example.yaml')
print(OmegaConf.to_yaml(conf))

server:
  port: 80
log:
  file: ???
  rotation: 3600
users:
- user1
- user2



#### Object style access:

In [9]:
conf.server.port

80

#### dictionary style access

In [10]:
conf['log']['rotation']

3600

#### items in list

In [11]:
conf.users[0]

'user1'

#### Changing existing keys

In [12]:
conf.server.port = 81

#### Adding new keys

In [13]:
conf.server.hostname = "localhost"

#### Adding a new dictionary

In [14]:
conf.database = {'hostname': 'database01', 'port': 3306}

#### providing default values

In [15]:
conf.get('missing_key', 'a default value')

'a default value'

#### Accessing mandatory values
Accessing fields with the value *???* will cause a MissingMandatoryValue exception.
Use this to indicate that the value must be set before accessing.

In [16]:
import pytest
from omegaconf import MissingMandatoryValue

with pytest.raises(MissingMandatoryValue):
    conf.log.file

# Variable interpolation

OmegaConf support variable interpolation, Interpolations are evaluated lazily on access.

## Config node interpolation

Interpolations are evaluated lazily on field access.<br>
Note below that when printed the interpolations are not resolved.<br>
They get resolved once you access them.

In [17]:
conf = OmegaConf.load('../source/config_interpolation.yaml')
print(OmegaConf.to_yaml(conf))

server:
  host: localhost
  port: 80
client:
  url: http://${server.host}:${server.port}/
  server_port: ${server.port}



In [18]:
# Primitive interpolation types are inherited from the referenced value
print("conf.client.server_port: ", conf.client.server_port, type(conf.client.server_port).__name__)
# Composite interpolation types are always string
print("conf.client.url: ", conf.client.url, type(conf.client.url).__name__)

conf.client.server_port:  80 int
conf.client.url:  http://localhost:80/ str


`to_yaml()` will resolve interpolations if `resolve=True` is passed

In [19]:
print(OmegaConf.to_yaml(conf, resolve=True))

server:
  host: localhost
  port: 80
client:
  url: http://localhost:80/
  server_port: 80



Interpolations may be nested, enabling more advanced behavior like dynamically selecting a sub-config:

In [20]:
cfg = OmegaConf.create("""
plans:
    A: plan A
    B: plan B
selected_plan: A
plan: ${plans.${selected_plan}}
""")
print(f"Default: cfg.plan = {cfg.plan}")
cfg.selected_plan = "B"
print(f"After selecting plan B: cfg.plan = {cfg.plan}")

Default: cfg.plan = plan A
After selecting plan B: cfg.plan = plan B


## Environment variable interpolation

Environment variable interpolation is also supported.

In [21]:
# Let's set up the environment first (only needed for this demonstration)
import os
os.environ['USER'] = 'omry'

Here is an example config file interpolates with the USER environment variable:

In [22]:
conf = OmegaConf.load('../source/env_interpolation.yaml')
print(OmegaConf.to_yaml(conf))

user:
  name: ${env:USER}
  home: /home/${env:USER}



In [23]:
conf = OmegaConf.load('../source/env_interpolation.yaml')
print(OmegaConf.to_yaml(conf, resolve=True))

user:
  name: omry
  home: /home/omry



You can specify a default value to use in case the environment variable is not defined. The following example sets `abc123` as the the default value when `DB_PASSWORD` is not defined.

In [24]:
os.environ.pop('DB_PASSWORD', None)  # ensure env variable does not exist
cfg = OmegaConf.create({'database': {'password': '${env:DB_PASSWORD,abc123}'}})
print(repr(cfg.database.password))
OmegaConf.clear_cache(cfg) # clear resolver cache
os.environ["DB_PASSWORD"] = 'secret'
print(repr(cfg.database.password))

'abc123'
'secret'


Environment variables are parsed when they are recognized as valid quantities that may be evaluated (e.g., int, float, dict, list):

In [25]:
cfg = OmegaConf.create({'database': {'password': '${env:DB_PASSWORD,abc123}',
                                    'user': 'someuser',
                                    'port': '${env:DB_PORT,3306}',
                                    'nodes': '${env:DB_NODES,[]}'}})
os.environ["DB_PORT"] = '3308'
print(repr(cfg.database.port))  # converted to int
os.environ["DB_NODES"] = '[host1, host2, host3]'
print(repr(cfg.database.nodes))  # converted to list
os.environ["DB_PASSWORD"] = 'a%#@~{}$*&^?/<'
print(repr(cfg.database.password))  # kept as a string

3308
['host1', 'host2', 'host3']
'a%#@~{}$*&^?/<'


## Custom interpolations

You can add additional interpolation types using custom resolvers.
The example below creates a resolver that adds 10 to the given value.

In [26]:
OmegaConf.new_register_resolver("plus_10", lambda x: x + 10)
conf = OmegaConf.create({'key': '${plus_10:990}'})
conf.key

1000

You can take advantage of nested interpolations to perform custom operations over variables:

In [27]:
OmegaConf.new_register_resolver("plus", lambda x, y: x + y)
conf = OmegaConf.create({"a": 1, "b": 2, "a_plus_b": "${plus:${a},${b}}"})
conf.a_plus_b

3

By default a custom resolver’s output is cached, so that when it is called with the same inputs we always return the same value. This behavior may be disabled by setting `use_cache=False`:

In [28]:
import random
random.seed(1234)

OmegaConf.new_register_resolver("randint",
                                lambda a, b: random.randint(a, b))
c = OmegaConf.create({"x": "${randint:0, 1000}"})
print("With cache:")
print(f"c.x = {repr(c.x)}")
print(f"c.x = {repr(c.x)}")  # same as above thanks to the cache

random.seed(1234)
OmegaConf.new_register_resolver("randint_nocache",
                                lambda a, b: random.randint(a, b),
                                use_cache=False)  # <== disable cache!
c = OmegaConf.create({"x": "${randint_nocache:0, 1000}"})
print("Without cache:")
print(f"c.x = {repr(c.x)}")
print(f"c.x = {repr(c.x)}")  # not the same anymore since the cache is disabled

With cache:
c.x = 989
c.x = 989
Without cache:
c.x = 989
c.x = 796


# Merging configurations
Merging configurations enables the creation of reusable configuration files for each logical component instead of a single config file for each variation of your task.

Machine learning experiment example:
```python
conf = OmegaConf.merge(base_cfg, model_cfg, optimizer_cfg, dataset_cfg)
```

Web server configuration example:

```python
conf = OmegaConf.merge(server_cfg, plugin1_cfg, site1_cfg, site2_cfg)
```

The following example creates two configs from files, and one from the cli. It then combines them into a single object. Note how the port changes to 82, and how the users lists are combined.

In [29]:
base_conf = OmegaConf.load('../source/example2.yaml')
print(OmegaConf.to_yaml(base_conf))

server:
  port: 80
users:
- user1
- user2



In [30]:
second_conf = OmegaConf.load('../source/example3.yaml')
print(OmegaConf.to_yaml(second_conf))

log:
  file: log.txt



In [31]:
from omegaconf import OmegaConf
import sys

# Merge configs:
conf = OmegaConf.merge(base_conf, second_conf)

# Simulate command line arguments
sys.argv = ['program.py', 'server.port=82']
# Merge with cli arguments
conf.merge_with_cli()
print(OmegaConf.to_yaml(conf))

server:
  port: 82
users:
- user1
- user2
log:
  file: log.txt

